In [13]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from datetime import datetime
import os
import wandb
from pathlib import Path

# 본 과제 제출자는 현재 우분투 도커 환경에서 작업중이므로 다음과 같이 경로 설정
BASE_PATH="/home/Deep-Learning-study"
import sys
sys.path.append(BASE_PATH)

CURRENT_FILE_PATH = os.getcwd()
CHECKPOINT_FILE_PATH = os.path.join(CURRENT_FILE_PATH, "checkpoints")

if not os.path.isdir(CHECKPOINT_FILE_PATH):
  os.makedirs(os.path.join(CURRENT_FILE_PATH, "checkpoints"))

In [14]:
from _01_code._15_lstm_and_its_application.f_arg_parser import get_parser
from _01_code._14_rnn.g_rnn_trainer import RegressionTrainer
from _01_code._03_real_world_data_to_tensors.p_cryptocurrency_dataset_dataloader import get_cryptocurrency_data, \
  CryptoCurrencyDataset

In [15]:
def get_btc_krw_data(sequence_size=10, validation_size=100, test_size=10, is_regression=True):
    # 비트코인 데이터 로드 및 전처리
    X_train, X_validation, X_test, y_train, y_validation, y_test, y_train_date, y_validation_date, y_test_date \
        = get_cryptocurrency_data(
            sequence_size=sequence_size,        # 시계열 데이터의 시퀀스 길이
            validation_size=validation_size,    # 검증 데이터셋 크기
            test_size=test_size,               # 테스트 데이터셋 크기
            target_column='Close',             # 예측할 대상 컬럼 (종가)
            y_normalizer=1.0e7,                # 타겟값 정규화를 위한 스케일링 factor
            is_regression=is_regression         # 회귀/분류 태스크 구분 플래그
        )

    # PyTorch Dataset 객체 생성
    train_crypto_currency_dataset = CryptoCurrencyDataset(X=X_train, y=y_train)
    validation_crypto_currency_dataset = CryptoCurrencyDataset(X=X_validation, y=y_validation)
    test_crypto_currency_dataset = CryptoCurrencyDataset(X=X_test, y=y_test)

    # DataLoader 생성
    # 학습 데이터 로더: 배치 크기만큼 데이터를 랜덤하게 샘플링
    train_data_loader = DataLoader(
        dataset=train_crypto_currency_dataset, 
        batch_size=wandb.config.batch_size,    # wandb에 설정된 배치 크기 사용
        shuffle=True                           # 데이터 섞기 활성화
    )
    
    # 검증 데이터 로더: 학습과 동일한 배치 크기 사용
    validation_data_loader = DataLoader(
        dataset=validation_crypto_currency_dataset, 
        batch_size=wandb.config.batch_size,
        shuffle=True
    )
    
    # 테스트 데이터 로더: 전체 테스트 데이터를 한 번에 처리
    test_data_loader = DataLoader(
        dataset=test_crypto_currency_dataset,
        batch_size=len(test_crypto_currency_dataset),  # 전체 테스트 데이터를 하나의 배치로
        shuffle=True
    )

    # 생성된 데이터 로더들 반환
    return train_data_loader, validation_data_loader, test_data_loader

In [16]:
def get_model():
    class MyModel(nn.Module):
        def __init__(self, n_input, n_output):
            super().__init__()

            # LSTM 레이어 정의
            self.lstm = nn.LSTM(
                input_size=n_input,     # 입력 특성의 개수 (5: Open, High, Low, Close, Volume)
                hidden_size=128,        # LSTM의 은닉 상태 크기
                num_layers=2,           # LSTM 레이어의 개수
                batch_first=True        # 입력 텐서의 형태를 (batch, seq, feature)로 지정
            )
            
            # 완전연결 레이어 정의
            self.fcn = nn.Linear(
                in_features=128,        # LSTM의 hidden_size와 동일
                out_features=n_output   # 출력값의 개수 (1: 다음 날의 종가)
            )

        def forward(self, x):
            # LSTM 레이어 통과
            # x 입력 shape: (batch_size, sequence_length, n_input)
            # hidden: (h_n, c_n) - LSTM의 최종 은닉 상태와 셀 상태
            x, hidden = self.lstm(x)
            
            # 마지막 시퀀스의 출력만 선택
            # x shape: (batch_size, sequence_length, hidden_size)
            # x[:, -1, :] shape: (batch_size, hidden_size)
            x = x[:, -1, :]
            
            # 선형 레이어를 통과시켜 최종 예측값 생성
            # 출력 shape: (batch_size, n_output)
            x = self.fcn(x)
            return x

    # 모델 인스턴스 생성
    my_model = MyModel(
        n_input=5,    # 입력 특성 수 (OHLCV)
        n_output=1    # 출력 특성 수 (다음 날 종가)
    )

    return my_model

In [17]:
def main(args):
    # 현재 시간을 문자열로 변환 (실행 식별자로 사용)
    run_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    # wandb 설정을 위한 하이퍼파라미터 딕셔너리 생성
    config = {
        'epochs': args.epochs,                    # 총 학습 에폭 수
        'batch_size': args.batch_size,           # 배치 크기
        'validation_intervals': args.validation_intervals,  # 검증 주기
        'learning_rate': args.learning_rate,      # 학습률
        'early_stop_patience': args.early_stop_patience,  # 조기 종료 인내 횟수
        'early_stop_delta': args.early_stop_delta,  # 조기 종료 임계값
        'weight_decay': args.weight_decay         # L2 정규화 계수
    }

    # wandb 프로젝트 초기화
    project_name = "lstm_regression_btc_krw"
    wandb.init(
        mode="online" if args.wandb else "disabled",  # wandb 사용 여부
        project=project_name,                         # 프로젝트 이름
        notes="btc_krw experiment with lstm",         # 실험 노트
        tags=["lstm", "regression", "btc_krw"],       # 태그
        name=run_time_str,                           # 실행 이름
        config=config                                # 설정값
    )
    # 설정값 출력
    print(args)
    print(wandb.config)

    # 데이터 로더 생성 (테스트 데이터 로더는 사용하지 않음)
    train_data_loader, validation_data_loader, _ = get_btc_krw_data()
    
    # GPU 사용 가능 여부 확인 및 디바이스 설정
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Training on device {device}.")

    # 모델 생성 및 지정된 디바이스로 이동
    model = get_model()
    model.to(device)
    wandb.watch(model)  # wandb로 모델 모니터링 시작

    # 옵티마이저 설정 (Adam)
    optimizer = optim.Adam(
        model.parameters(),
        lr=wandb.config.learning_rate,
        weight_decay=wandb.config.weight_decay
    )

    # 학습 객체 생성
    regression_trainer = RegressionTrainer(
        project_name=project_name,           # 프로젝트 이름
        model=model,                         # 학습할 모델
        optimizer=optimizer,                 # 옵티마이저
        train_data_loader=train_data_loader,  # 학습 데이터 로더
        validation_data_loader=validation_data_loader,  # 검증 데이터 로더
        test_data_loader=None,              # 테스트 데이터 로더 (사용하지 않음)
        run_time_str=run_time_str,          # 실행 식별자
        wandb=wandb,                        # wandb 객체
        device=device,                      # 학습 디바이스
        checkpoint_file_path=CHECKPOINT_FILE_PATH  # 체크포인트 저장 경로
    )
    
    # 학습 실행
    regression_trainer.train_loop()

    # wandb 실행 종료
    wandb.finish()

In [18]:
if __name__ == "__main__":
    import sys
    if 'ipykernel' in sys.modules:  # Jupyter Notebook에서 실행 중인지 확인
        # Jupyter에서 실행할 때는 기본값 사용
        class Args:
            def __init__(self):
                self.wandb = True
                self.batch_size = 32
                self.epochs = 100
                self.learning_rate = 0.001
                self.weight_decay = 0.0001
                self.validation_intervals = 1
                self.early_stop_patience = 5
                self.early_stop_delta = 0.001
        args = Args()
    else:
        # 일반 Python 스크립트로 실행할 때는 argparse 사용
        parser = get_parser()
        args = parser.parse_args()
    
    main(args)

{'epochs': 100, 'batch_size': 32, 'validation_intervals': 1, 'learning_rate': 0.001, 'early_stop_patience': 5, 'early_stop_delta': 0.001, 'weight_decay': 0.0001}
Training on device cuda:0.
[Epoch   1] T_loss: 1.40876, V_loss: 1.63254, Early stopping is stated! | T_time: 00:00:00, T_speed: 0.003
[Epoch   2] T_loss: 0.05143, V_loss: 1.78813, Early stopping counter: 1 out of 5 | T_time: 00:00:00, T_speed: 0.004
[Epoch   3] T_loss: 0.03782, V_loss: 0.86087, V_loss decreased (1.63254 --> 0.86087). Saving model... | T_time: 00:00:00, T_speed: 0.004
[Epoch   4] T_loss: 0.03518, V_loss: 0.85138, V_loss decreased (0.86087 --> 0.85138). Saving model... | T_time: 00:00:00, T_speed: 0.004
[Epoch   5] T_loss: 0.02939, V_loss: 0.76669, V_loss decreased (0.85138 --> 0.76669). Saving model... | T_time: 00:00:01, T_speed: 0.024
[Epoch   6] T_loss: 0.02597, V_loss: 0.63942, V_loss decreased (0.76669 --> 0.63942). Saving model... | T_time: 00:00:01, T_speed: 0.014
[Epoch   7] T_loss: 0.02391, V_loss: 0.6

Epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
Training loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Training speed (epochs/sec.),▁▁▁▁▂▁▁▁▃▂▁▁█▂▂▂▁▃▂▂▂▄▂▂▂▆▃
Validation loss,▇█▄▄▃▃▃▂▃▃▃▁▂▁▁▁▂▁▁▁▁▁▃▁▁▁▁
Epoch,27
Training loss,0.01493
Training speed (epochs/sec.),0.07359
Validation loss,0.34908
